In [163]:
import xgboost as xgb
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
import dataframe_image as dfi
import matplotlib.pyplot as plt
import sklearn.metrics as mtr
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pickle


# Primeira Execução

In [164]:
df = pd.read_csv('rota33642.csv')
df.head()

,data_partida,data_chegada,linha,tempo_viagem,equipamento,dia_semana,partidaTimeStamp,qtdDiasAno,direcao,tipo_dia,turno_dia,chegadaTimeStamp,hora,hora_dia
0,2019-07-02 05:35:51,2019-07-02 05:47:44,33642,713,1083,2,1562056551,183,2,1,2,1562057264,NaN,5
1,2019-07-02 06:11:07,2019-07-02 06:25:15,33642,848,1083,2,1562058667,183,2,1,2,1562059515,NaN,6
2,2019-07-02 06:53:26,2019-07-02 07:08:02,33642,876,1083,2,1562061206,183,2,1,2,1562062082,NaN,6
3,2019-07-02 07:36:44,2019-07-02 07:55:03,33642,1099,1083,2,1562063804,183,2,1,2,1562064903,NaN,7
4,2019-07-02 08:16:37,2019-07-02 08:30:55,33642,858,1083,2,1562066197,183,2,1,2,1562067055,NaN,8


In [165]:
# Dropamos:
# A data_chegada e a chegadaTimeStamp pois com esses valores daria para prever o tempo de viagem(Y)
# linha e equipamento por se tratar apenas de IDs
# hora por conter apenas valores nulos
# direcao por conter apenas o valor 2

df = df.drop(['data_chegada','linha','equipamento','direcao','chegadaTimeStamp','hora'], axis=1)

In [166]:
corr = df.corr()
corr

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2438084875.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,tempo_viagem,dia_semana,partidaTimeStamp,qtdDiasAno,tipo_dia,turno_dia,hora_dia
tempo_viagem,1.000000,-0.091237,0.036642,0.033283,-0.041862,0.360651,0.375852
dia_semana,-0.091237,1.000000,0.018559,0.018371,0.044735,0.018066,0.020579
partidaTimeStamp,0.036642,0.018559,1.000000,0.999960,-0.012245,0.006308,0.005832
qtdDiasAno,0.033283,0.018371,0.999960,1.000000,-0.012287,-0.002152,-0.003087
tipo_dia,-0.041862,0.044735,-0.012245,-0.012287,1.000000,0.001882,0.004642
turno_dia,0.360651,0.018066,0.006308,-0.002152,0.001882,1.000000,0.948907
hora_dia,0.375852,0.020579,0.005832,-0.003087,0.004642,0.948907,1.000000


In [167]:
df[['hora_dia','turno_dia']].corr()

,hora_dia,turno_dia
hora_dia,1.000000,0.948907
turno_dia,0.948907,1.000000


In [168]:
df[['partidaTimeStamp','qtdDiasAno']].corr()

,partidaTimeStamp,qtdDiasAno
partidaTimeStamp,1.00000,0.99996
qtdDiasAno,0.99996,1.00000


In [169]:
#Vamos tirar hora_dia pois tem uma alta correlação com turno dia, escolhemos essa coluna pois os valores são apenas de 1 a 4 enquanto hora é de 0 a 23
df = df.drop(['qtdDiasAno'], axis=1)

In [170]:
#Ordenando pela Data
df = df.sort_values(by=["data_partida"], ascending=False)

In [171]:
#Transformamos data_partida em index
df.index = df['data_partida']
df = df.drop(['data_partida'], axis=1)

In [172]:
#Separamos treino e teste
shape = int(df.shape[0] * 0.3)
df_teste = df[:shape]
df_treino = df[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (6520, 6)      Teste:(2794, 6)


In [173]:
# Separamos a classe dos atributos 
x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
y_teste = df_teste[['tempo_viagem']]
x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
y_treino = df_treino[['tempo_viagem']]

In [174]:
params_xgb = {
        "n_estimators": list(range(100, 1100, 100)), #Number of gradient boosted trees. Equivalent to number of boosting rounds
        "max_depth": list(range(2, 15)),#Maximum tree depth for base learners.
        "min_child_weight": list(range(1, 11)),#Minimum sum of instance weight(hessian) needed in a child.
        "learning_rate": [0.3, 0.2, 0.1, 0.05, 0.01, 0.005],#Boosting learning rate (xgb’s “eta”)
        "gamma": np.arange(0, 0.7, 0.1)#Minimum loss reduction required to make a further partition on a leaf node of the tree.
        }

# Hyperparâmetros do Random Forest
params_rf = {
        "n_estimators": list(range(100, 1100, 100)),#The number of trees in the forest.
        "bootstrap": [True, False],#Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.
        "max_depth": list(range(2, 15)),#The maximum depth of the tree.
        "max_features": [1.0, "sqrt", "log2"],#The number of features to consider when looking for the best split:
        "min_samples_leaf": list(range(1, 11)),#The minimum number of samples required to split an internal node
        "min_samples_split": list(range(2, 11)),#The minimum number of samples required to be at a leaf node.
        }

In [175]:
xgb_rand_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stop_rounds=100, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.005, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, ...)

In [176]:
rf_rand_search.best_estimator_

RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                      min_samples_split=7, n_estimators=900)

In [233]:
#Criação e fit do modelo XGB
start_xgb = datetime.now()
modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stop_rounds=100, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.005, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=5, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0)
#modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
#xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#xgb_rand_search.fit(x_treino, y_treino)
#modelo_xgb = xgb_rand_search.best_estimator_
modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
end_xgb = datetime.now()

#Criação e fit do modelo Random Forest
start_rf = datetime.now()
modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                      min_samples_split=7, n_estimators=900)
#modelo_rf = RandomForestRegressor()
#rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#rf_rand_search.fit(x_treino, y_treino)
#modelo_rf = rf_rand_search.best_estimator_
modelo_rf.fit(x_treino, y_treino)
end_rf = datetime.now()

#Criação e fit do modelo Linear Regression
start_lr = datetime.now()
modelo_lr = LinearRegression()
modelo_lr.fit(x_treino, y_treino)
end_lr = datetime.now()


tempo_xgb = end_xgb - start_xgb
tempo_rf = end_rf - start_rf
tempo_lr = end_lr - start_lr

[20:54:34] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1179.68362
[1]	validation_0-rmse:1173.91704
[2]	validation_0-rmse:1168.17990
[3]	validation_0-rmse:1162.47206
[4]	validation_0-rmse:1156.79335
[5]	validation_0-rmse:1151.14365
[6]	validation_0-rmse:1145.52278
[7]	validation_0-rmse:1139.93066
[8]	validation_0-rmse:1134.36707
[9]	validation_0-rmse:1128.83196
[10]	validation_0-rmse:1123.32516
[11]	validation_0-rmse:1117.84648
[12]	validation_0-rmse:1112.39585
[13]	validation_0-rmse:1106.97311
[14]	validation_0-rmse:1101.57809
[15]	validation_0-rmse:1096.21069
[16]	validation_0-rmse:1090.87074
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2675320578.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)


In [178]:
# Fizemos as predições e juntamos no DataFrame
df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
df_teste["predicao_rf"] = modelo_rf.predict(x_teste)

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\1876206521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\1876206521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\1876206521.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [179]:
# Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_xgb"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_xgb"])

RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_rf"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_rf"])

RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_lr"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_lr"])

In [180]:
metrics = []
metrics.append(['Primeira execução','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
metrics.append(['Primeira execução','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
metrics.append(['Primeira execução','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 
metrics = pd.DataFrame(metrics, columns=['Descrição','Métrica', 'XGBoosting','Random Forest','Linear Regression'])
metrics.to_csv('metrica_primeira_execução.csv',index=False)
metrics

,Descrição,Métrica,XGBoosting,Random Forest,Linear Regression
0,Primeira execução,RRSE,0.692901,0.717718,0.939445
1,Primeira execução,MAPE,0.103641,0.108865,0.153585
2,Primeira execução,Tempo,0:00:02.886090,0:00:02.755531,0:00:00.003961


In [181]:
df_teste.to_csv('previsão_primeira_execução.csv',index=False)
df_teste.head()

,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,predicao_xgb,predicao_lr,predicao_rf
data_partida,,,,,,,,,
2019-10-01 00:40:28,680,2,1569901228,1,1,0,831.875977,978.708272,839.231428
2019-10-01 00:16:27,849,2,1569899787,1,1,0,831.875977,978.703918,839.231428
2019-10-01 00:00:45,852,2,1569898845,1,1,0,831.875977,978.701072,839.231428
2019-09-30 23:46:49,824,1,1569898009,1,4,23,1042.120117,1373.619877,1121.457373
2019-09-30 23:30:14,993,1,1569897014,1,4,23,1042.120117,1373.616871,1121.457373


# Teste com Clima

In [182]:
#Lendo dados de clima e convertendo a data pra datetime (na leitura veio como objeto)
clima = pd.read_csv('clima.csv')
clima['Datetime'] = pd.to_datetime(clima['Datetime'])

In [183]:
#copiando o DAtaframe original para não alterar o primeiro
aux = df.copy()
#Mudanças na data_partida para conseguir fazer o merge
aux["data_partida2"] = pd.to_datetime(aux.index)
aux["data_partida2"] = aux["data_partida2"].dt.round("H")
#Merge dos dados e exclusão da coluna auxiliar
df_clima = aux.merge(clima, left_on="data_partida2", right_on="Datetime", how="left")
df_clima.drop(['data_partida2','Datetime'],axis=1,inplace=True)
df_clima.index = aux.index
df_clima.head()

,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,Descricao Chuva,Calor
data_partida,,,,,,,,
2019-10-01 00:40:28,680,2,1569901228,1,1,0,0,1
2019-10-01 00:16:27,849,2,1569899787,1,1,0,0,1
2019-10-01 00:00:45,852,2,1569898845,1,1,0,0,1
2019-09-30 23:46:49,824,1,1569898009,1,4,23,0,1
2019-09-30 23:30:14,993,1,1569897014,1,4,23,0,1


In [184]:
#Separamos treino e teste
shape = int(df_clima.shape[0] * 0.3)
df_teste = df_clima[:shape]
df_treino = df_clima[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (6520, 8)      Teste:(2794, 8)


In [185]:
# Separamos a classe dos atributos 
x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia','Descricao Chuva','Calor']]
y_teste = df_teste[['tempo_viagem']]
x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia','Descricao Chuva','Calor']]
y_treino = df_treino[['tempo_viagem']]

In [186]:
#Criação e fit do modelo XGB
start_xgb = datetime.now()
modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stop_rounds=100, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.005, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=5, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0)
#modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
#xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#xgb_rand_search.fit(x_treino, y_treino)
#modelo_xgb = xgb_rand_search.best_estimator_
modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
end_xgb = datetime.now()

#Criação e fit do modelo Random Forest
start_rf = datetime.now()
modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                      min_samples_split=7, n_estimators=900)
#modelo_rf = RandomForestRegressor()
#rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
#rf_rand_search.fit(x_treino, y_treino)
#modelo_rf = rf_rand_search.best_estimator_
modelo_rf.fit(x_treino, y_treino)
end_rf = datetime.now()

#Criação e fit do modelo Linear Regression
start_lr = datetime.now()
modelo_lr = LinearRegression()
modelo_lr.fit(x_treino, y_treino)
end_lr = datetime.now()


tempo_xgb = end_xgb - start_xgb
tempo_rf = end_rf - start_rf
tempo_lr = end_lr - start_lr

[20:27:03] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1177.13656
[1]	validation_0-rmse:1171.38344
[2]	validation_0-rmse:1165.65965
[3]	validation_0-rmse:1159.96504
[4]	validation_0-rmse:1154.29961
[5]	validation_0-rmse:1148.66303
[6]	validation_0-rmse:1143.05529
[7]	validation_0-rmse:1137.47617
[8]	validation_0-rmse:1131.92554
[9]	validation_0-rmse:1126.40328
[10]	validation_0-rmse:1120.90924
[11]	validation_0-rmse:1115.44327
[12]	validation_0-rmse:1110.00525
[13]	validation_0-rmse:1104.59506
[14]	validation_0-rmse:1099.21252
[15]	validation_0-rmse:1093.85753
[16]	validation_0-rmse:1088.52996
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2675320578.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)


In [187]:
# Fizemos as predições e juntamos no DataFrame
df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
df_teste["predicao_rf"] = modelo_rf.predict(x_teste)

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\1876206521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\1876206521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\1876206521.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [188]:
# Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_xgb"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_xgb"])

RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_rf"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_rf"])

RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste["predicao_lr"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste["predicao_lr"])

In [189]:
metrics = []
metrics.append(['Teste Clima','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
metrics.append(['Teste Clima','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
metrics.append(['Teste Clima','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 
metrics = pd.DataFrame(metrics, columns=['Descrição','Métrica', 'XGBoosting','Random Forest','Linear Regression'])
metrics.to_csv('metrica_teste_clime.csv',index=False)
metrics

,Descrição,Métrica,XGBoosting,Random Forest,Linear Regression
0,Teste Clima,RRSE,0.6943,0.725259,0.916085
1,Teste Clima,MAPE,0.103104,0.109501,0.151943
2,Teste Clima,Tempo,0:00:02.885372,0:00:02.388405,0:00:00.003001


In [190]:
df_teste.to_csv('previsão_teste_clima.csv',index=False)
df_teste.head()

,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,Descricao Chuva,Calor,predicao_xgb,predicao_lr,predicao_rf
data_partida,,,,,,,,,,,
2019-10-01 00:40:28,680,2,1569901228,1,1,0,0,1,841.116333,1026.636094,880.829429
2019-10-01 00:16:27,849,2,1569899787,1,1,0,0,1,841.116333,1026.633915,880.829429
2019-10-01 00:00:45,852,2,1569898845,1,1,0,0,1,841.116333,1026.632491,880.829429
2019-09-30 23:46:49,824,1,1569898009,1,4,23,0,1,1021.690552,1371.491567,1077.335329
2019-09-30 23:30:14,993,1,1569897014,1,4,23,0,1,1021.690552,1371.490062,1077.335329


# Tempos Anteriores

In [191]:
df_tempos = df.copy()
df_tempos["tempo_viagem_1"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_2"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_3"] = df_tempos["tempo_viagem"]
df_tempos["tempo_viagem_4"] = df_tempos["tempo_viagem"]

df_tempos["tempo_viagem_1"] = df_tempos["tempo_viagem_1"].shift(1)
df_tempos["tempo_viagem_2"] = df_tempos["tempo_viagem_2"].shift(2)
df_tempos["tempo_viagem_3"] = df_tempos["tempo_viagem_3"].shift(3)
df_tempos["tempo_viagem_4"] = df_tempos["tempo_viagem_4"].shift(4)
df_tempos.head()

,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,tempo_viagem_1,tempo_viagem_2,tempo_viagem_3,tempo_viagem_4
data_partida,,,,,,,,,,
2019-10-01 00:40:28,680,2,1569901228,1,1,0,NaN,NaN,NaN,NaN
2019-10-01 00:16:27,849,2,1569899787,1,1,0,680.0,NaN,NaN,NaN
2019-10-01 00:00:45,852,2,1569898845,1,1,0,849.0,680.0,NaN,NaN
2019-09-30 23:46:49,824,1,1569898009,1,4,23,852.0,849.0,680.0,NaN
2019-09-30 23:30:14,993,1,1569897014,1,4,23,824.0,852.0,849.0,680.0


In [192]:
shape = int(df_tempos.shape[0] * 0.3)
df_teste = df_tempos[:shape]
df_treino = df_tempos[shape:]
print(f'Treino: {df_treino.shape}      Teste:{df_teste.shape}')

Treino: (6520, 10)      Teste:(2794, 10)


In [194]:
metrics = []
for cenario in range(1, 5):
        if cenario == 1:
                df_teste = df_teste.iloc[1:]
                df_treino = df_treino.iloc[1:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"]]
                y_treino = df_treino["tempo_viagem"]

        elif cenario == 2:
                df_teste = df_teste.iloc[2:]
                df_treino = df_treino.iloc[2:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]]
                y_treino = df_treino["tempo_viagem"]

        elif cenario == 3:
                df_teste = df_teste.iloc[3:]
                df_treino = df_treino.iloc[3:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"]]
                y_treino = df_treino["tempo_viagem"]
        else:
                df_teste = df_teste.iloc[4:]
                df_treino = df_treino.iloc[4:]

                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3","tempo_viagem_4"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3","tempo_viagem_4"]]
                y_treino = df_treino["tempo_viagem"]

        #Criação e fit do modelo XGB
        start_xgb = datetime.now()
        modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
                early_stop_rounds=100, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
                grow_policy='depthwise', importance_type=None,
                interaction_constraints='', learning_rate=0.005, max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
                min_child_weight=5, monotone_constraints='()',
                n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
                random_state=0, reg_alpha=0)
        #modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
        #xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #xgb_rand_search.fit(x_treino, y_treino)
        #modelo_xgb = xgb_rand_search.best_estimator_
        modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
        end_xgb = datetime.now()

        #Criação e fit do modelo Random Forest
        start_rf = datetime.now()
        modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                        min_samples_split=7, n_estimators=900)
        #modelo_rf = RandomForestRegressor()
        #rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #rf_rand_search.fit(x_treino, y_treino)
        #modelo_rf = rf_rand_search.best_estimator_
        modelo_rf.fit(x_treino, y_treino)
        end_rf = datetime.now()

        #Criação e fit do modelo Linear Regression
        start_lr = datetime.now()
        modelo_lr = LinearRegression()
        modelo_lr.fit(x_treino, y_treino)
        end_lr = datetime.now()


        tempo_xgb = end_xgb - start_xgb
        tempo_rf = end_rf - start_rf
        tempo_lr = end_lr - start_lr

        # Fizemos as predições e juntamos no DataFrame
        df_teste[f"predicao_xgb_{cenario}"] = modelo_xgb.predict(x_teste)
        df_teste[f"predicao_lr_{cenario}"] = modelo_lr.predict(x_teste)
        df_teste[f"predicao_rf_{cenario}"] = modelo_rf.predict(x_teste)

        # Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
        RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_xgb_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_xgb_{cenario}"])

        RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_rf_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_rf_{cenario}"])

        RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_lr_{cenario}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_lr_{cenario}"])

        metrics.append([f'Teste Tempos {cenario}','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
        metrics.append([f'Teste Tempos {cenario}','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
        metrics.append([f'Teste Tempos {cenario}','Tempo', tempo_xgb, tempo_rf, tempo_lr]) 

[20:29:11] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1177.17536
[1]	validation_0-rmse:1171.43010
[2]	validation_0-rmse:1165.71426
[3]	validation_0-rmse:1160.02759
[4]	validation_0-rmse:1154.36845
[5]	validation_0-rmse:1148.73828
[6]	validation_0-rmse:1143.13834
[7]	validation_0-rmse:1137.56556
[8]	validation_0-rmse:1132.02124
[9]	validation_0-rmse:1126.50531
[10]	validation_0-rmse:1121.01761
[11]	validation_0-rmse:1115.55945
[12]	validation_0-rmse:1110.12774
[13]	validation_0-rmse:1104.71747
[14]	validation_0-rmse:1099.33489
[15]	validation_0-rmse:1093.97982
[16]	validation_0-rmse:1088.65845
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb_{cenario}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_lr_{cenario}"] = modelo_lr.predict(x_teste)


[20:29:18] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1177.17350
[1]	validation_0-rmse:1171.42439
[2]	validation_0-rmse:1165.70226
[3]	validation_0-rmse:1160.01154
[4]	validation_0-rmse:1154.34749
[5]	validation_0-rmse:1148.71461
[6]	validation_0-rmse:1143.10812
[7]	validation_0-rmse:1137.53248
[8]	validation_0-rmse:1131.98297
[9]	validation_0-rmse:1126.46405
[10]	validation_0-rmse:1120.97096
[11]	validation_0-rmse:1115.50819
[12]	validation_0-rmse:1110.07096
[13]	validation_0-rmse:1104.66378
[14]	validation_0-rmse:1099.28189
[15]	validation_0-rmse:1093.92753
[16]	validation_0-rmse:1088.60265
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_rf_{cenario}"] = modelo_rf.predict(x_teste)


[62]	validation_0-rmse:870.89656
[63]	validation_0-rmse:866.71462
[64]	validation_0-rmse:862.55347
[65]	validation_0-rmse:858.41435
[66]	validation_0-rmse:854.29690
[67]	validation_0-rmse:850.19999
[68]	validation_0-rmse:846.12436
[69]	validation_0-rmse:842.07042
[70]	validation_0-rmse:838.03733
[71]	validation_0-rmse:834.02530
[72]	validation_0-rmse:830.03339
[73]	validation_0-rmse:826.06284
[74]	validation_0-rmse:822.11169
[75]	validation_0-rmse:818.18202
[76]	validation_0-rmse:814.27271
[77]	validation_0-rmse:810.38212
[78]	validation_0-rmse:806.51230
[79]	validation_0-rmse:802.66363
[80]	validation_0-rmse:798.83391
[81]	validation_0-rmse:795.02155
[82]	validation_0-rmse:791.23254
[83]	validation_0-rmse:787.46219
[84]	validation_0-rmse:783.71253
[85]	validation_0-rmse:779.97869
[86]	validation_0-rmse:776.26752
[87]	validation_0-rmse:772.57493
[88]	validation_0-rmse:768.89908
[89]	validation_0-rmse:765.24594
[90]	validation_0-rmse:761.61076
[91]	validation_0-rmse:757.99565
[92]	valid

...

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb_{cenario}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_lr_{cenario}"] = modelo_lr.predict(x_teste)


[20:29:32] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1177.25703
[1]	validation_0-rmse:1171.50401
[2]	validation_0-rmse:1165.78034
[3]	validation_0-rmse:1160.08840
[4]	validation_0-rmse:1154.42281
[5]	validation_0-rmse:1148.78869
[6]	validation_0-rmse:1143.18063
[7]	validation_0-rmse:1137.60117
[8]	validation_0-rmse:1132.05261
[9]	validation_0-rmse:1126.52984
[10]	validation_0-rmse:1121.03766
[11]	validation_0-rmse:1115.57101
[12]	validation_0-rmse:1110.13231
[13]	validation_0-rmse:1104.72361
[14]	validation_0-rmse:1099.34020
[15]	validation_0-rmse:1093.98655
[16]	validation_0-rmse:1088.65788
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_rf_{cenario}"] = modelo_rf.predict(x_teste)


[60]	validation_0-rmse:879.26983
[61]	validation_0-rmse:875.03848
[62]	validation_0-rmse:870.83210
[63]	validation_0-rmse:866.64565
[64]	validation_0-rmse:862.48080
[65]	validation_0-rmse:858.33606
[66]	validation_0-rmse:854.21577
[67]	validation_0-rmse:850.11511
[68]	validation_0-rmse:846.03554
[69]	validation_0-rmse:841.97878
[70]	validation_0-rmse:837.94293
[71]	validation_0-rmse:833.92602
[72]	validation_0-rmse:829.93013
[73]	validation_0-rmse:825.95684
[74]	validation_0-rmse:822.00122
[75]	validation_0-rmse:818.06898
[76]	validation_0-rmse:814.15474
[77]	validation_0-rmse:810.26116
[78]	validation_0-rmse:806.38967
[79]	validation_0-rmse:802.53533
[80]	validation_0-rmse:798.70108
[81]	validation_0-rmse:794.88663
[82]	validation_0-rmse:791.09278
[83]	validation_0-rmse:787.31783
[84]	validation_0-rmse:783.56463
[85]	validation_0-rmse:779.82921
[86]	validation_0-rmse:776.11350
[87]	validation_0-rmse:772.41883
[88]	validation_0-rmse:768.74069
[89]	validation_0-rmse:765.08180
[90]	valid

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb_{cenario}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_lr_{cenario}"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\2273390743.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [195]:
metrics = pd.DataFrame(metrics, columns=['Descrição','Métrica', 'XGBoosting','Random Forest','Linear Regression'])
metrics.to_csv('metrica_teste_tempos.csv',index=False)
metrics


,Descrição,Métrica,XGBoosting,Random Forest,Linear Regression
0,Teste Tempos 1,RRSE,0.702842,0.70939,0.800414
1,Teste Tempos 1,MAPE,0.104555,0.106104,0.127016
2,Teste Tempos 1,Tempo,0:00:02.968255,0:00:03.470308,0:00:00.002031
3,Teste Tempos 2,RRSE,0.689658,0.69076,0.743665
4,Teste Tempos 2,MAPE,0.103109,0.103605,0.116628
5,Teste Tempos 2,Tempo,0:00:02.751587,0:00:03.823341,0:00:00.003002
6,Teste Tempos 3,RRSE,0.686471,0.685831,0.724371
7,Teste Tempos 3,MAPE,0.102873,0.103496,0.113235
8,Teste Tempos 3,Tempo,0:00:02.700534,0:00:04.092818,0:00:00.003000
9,Teste Tempos 4,RRSE,0.68553,0.678678,0.713824


In [196]:
df_teste.to_csv('previsão_teste_tempos.csv',index=False)
df_teste.head()

,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,tempo_viagem_1,tempo_viagem_2,tempo_viagem_3,tempo_viagem_4,...,predicao_rf_1,predicao_xgb_2,predicao_lr_2,predicao_rf_2,predicao_xgb_3,predicao_lr_3,predicao_rf_3,predicao_xgb_4,predicao_lr_4,predicao_rf_4
data_partida,,,,,,,,,,,,,,,,,,,,,
2019-09-30 21:50:37,1428,1,1569891037,1,4,21,1055.0,950.0,932.0,1063.0,...,1139.967365,1037.835938,1119.977495,1064.076844,1029.365601,1089.852324,1044.099966,1059.065430,1102.497244,1064.158759
2019-09-30 21:45:34,986,1,1569890734,1,4,21,1428.0,1055.0,950.0,932.0,...,1169.438421,1097.359497,1286.553074,1138.225296,1094.219116,1219.115414,1112.622856,1090.103027,1181.472368,1099.967070
2019-09-30 21:32:06,1086,1,1569889926,1,4,21,986.0,1428.0,1055.0,950.0,...,1134.304907,1090.786255,1259.261626,1188.017823,1090.396973,1220.250922,1139.239015,1107.109741,1181.511385,1094.356703
2019-09-30 21:21:11,1287,1,1569889271,1,4,21,1086.0,986.0,1428.0,1055.0,...,1145.991733,1059.919312,1143.146292,1073.654403,1149.147461,1224.669432,1185.415233,1161.128296,1203.119032,1192.630275
2019-09-30 21:10:22,956,1,1569888622,1,4,21,1287.0,1086.0,986.0,1428.0,...,1146.783318,1085.977905,1247.756401,1125.508216,1075.305786,1198.072692,1101.208286,1130.301514,1259.482670,1123.312637


# Testes Intervalos de Teste_Treino

In [228]:
#Separamos treino e teste
df_intervalos = df.copy()
df_intervalos["data_partida2"] = pd.to_datetime(df_intervalos.index)

df_teste.head()

,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,data_partida2,predicao_xgb 14 79,predicao_lr 14 79,predicao_rf 14 79
data_partida,,,,,,,,,,
2019-10-01 00:40:28,680,2,1569901228,1,1,0,2019-10-01 00:40:28,861.536194,617.638155,887.929132
2019-10-01 00:16:27,849,2,1569899787,1,1,0,2019-10-01 00:16:27,861.536194,617.699797,887.929132
2019-10-01 00:00:45,852,2,1569898845,1,1,0,2019-10-01 00:00:45,861.536194,617.740094,887.929132
2019-09-30 23:46:49,824,1,1569898009,1,4,23,2019-09-30 23:46:49,974.830933,1043.810941,927.863931
2019-09-30 23:30:14,993,1,1569897014,1,4,23,2019-09-30 23:30:14,974.830933,1043.853505,927.863931


In [229]:
data_atual = df_intervalos["data_partida2"].dt.date.min() + timedelta(days=14)
dias_treino = 14
dias_teste = 79


metrics = []
while data_atual < (df_intervalos["data_partida2"].dt.date.max() - timedelta(days=14)):
        df_teste = df_intervalos[df_intervalos["data_partida2"].dt.date > data_atual]
        df_treino = df_intervalos[df_intervalos["data_partida2"].dt.date <= data_atual]

        #Separamos a classe dos atributos 
        x_teste = df_teste[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
        y_teste = df_teste[['tempo_viagem']]
        x_treino = df_treino[['dia_semana','partidaTimeStamp','tipo_dia','hora_dia']]
        y_treino = df_treino[['tempo_viagem']]

        #Criação e fit do modelo XGB
        start_xgb = datetime.now()
        modelo_xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
                early_stop_rounds=100, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, gamma=0.0, gpu_id=-1,
                grow_policy='depthwise', importance_type=None,
                interaction_constraints='', learning_rate=0.005, max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
                min_child_weight=5, monotone_constraints='()',
                n_estimators=900, n_jobs=0, num_parallel_tree=1, predictor='auto',
                random_state=0, reg_alpha=0)
        #modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
        #xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #xgb_rand_search.fit(x_treino, y_treino)
        #modelo_xgb = xgb_rand_search.best_estimator_
        modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)],verbose=False)
        end_xgb = datetime.now()

        #Criação e fit do modelo Random Forest
        start_rf = datetime.now()
        modelo_rf = RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=4,
                        min_samples_split=7, n_estimators=900)
        #modelo_rf = RandomForestRegressor()
        #rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        #rf_rand_search.fit(x_treino, y_treino)
        #modelo_rf = rf_rand_search.best_estimator_
        modelo_rf.fit(x_treino, y_treino)
        end_rf = datetime.now()

        #Criação e fit do modelo Linear Regression
        start_lr = datetime.now()
        modelo_lr = LinearRegression()
        modelo_lr.fit(x_treino, y_treino)
        end_lr = datetime.now()


        tempo_xgb = end_xgb - start_xgb
        tempo_rf = end_rf - start_rf
        tempo_lr = end_lr - start_lr

        # Fizemos as predições e juntamos no DataFrame 
        df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
        df_teste[f"predicao_lr {dias_treino} {dias_teste}"] = modelo_lr.predict(x_teste)
        df_teste[f"predicao_rf {dias_treino} {dias_teste}"] = modelo_rf.predict(x_teste)

        # Pesquisar os motivos de se usar apenas métricas relativas - Flávia disse que provavelmente por conta de outlier/valor distantes 
        RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_xgb {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_xgb {dias_treino} {dias_teste}"])

        RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_rf {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_rf {dias_treino} {dias_teste}"])

        RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"predicao_lr {dias_treino} {dias_teste}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
        MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"predicao_lr {dias_treino} {dias_teste}"])

        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','RRSE', RRSE_xgb, RRSE_rf, RRSE_lr])
        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','MAPE', MAPE_xgb, MAPE_rf, MAPE_lr]) 
        metrics.append([f'Teste Intervalos {dias_treino} {dias_teste}','Tempo', tempo_xgb, tempo_rf, tempo_lr])

        df_intervalos = df_intervalos.merge(df_teste[[f'predicao_xgb {dias_treino} {dias_teste}',f'predicao_rf {dias_treino} {dias_teste}',f'predicao_lr {dias_treino} {dias_teste}']], how="left", left_index=True, right_index=True)
        

        dias_teste -= 14
        dias_treino += 14

        data_atual += timedelta(days=14)



[20:50:24] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1168.81347
[1]	validation_0-rmse:1163.12869
[2]	validation_0-rmse:1157.47293
[3]	validation_0-rmse:1151.84600
[4]	validation_0-rmse:1146.24779
[5]	validation_0-rmse:1140.67810
[6]	validation_0-rmse:1135.13684
[7]	validation_0-rmse:1129.62387
[8]	validation_0-rmse:1124.13900
[9]	validation_0-rmse:1118.68213
[10]	validation_0-rmse:1113.25317
[11]	validation_0-rmse:1107.85190
[12]	validation_0-rmse:1102.47825
[13]	validation_0-rmse:1097.13195
[14]	validation_0-rmse:1091.81307
[15]	validation_0-rmse:1086.52129
[16]	validation_0-rmse:1081.25657
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

[20:50:28] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1161.43512
[1]	validation_0-rmse:1155.77162
[2]	validation_0-rmse:1150.13701
[3]	validation_0-rmse:1144.52514
[4]	validation_0-rmse:1138.94184
[5]	validation_0-rmse:1133.38701
[6]	validation_0-rmse:1127.86056
[7]	validation_0-rmse:1122.36226
[8]	validation_0-rmse:1116.89204
[9]	validation_0-rmse:1111.44972
[10]	validation_0-rmse:1106.03516
[11]	validation_0-rmse:1100.64829
[12]	validation_0-rmse:1095.28894
[13]	validation_0-rmse:1089.95695
[14]	validation_0-rmse:1084.65218
[15]	validation_0-rmse:1079.37451
[16]	validation_0-rmse:1074.12379
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_rf {dias_treino} {dias_teste}"] = modelo_rf.predict(x_teste)


[77]	validation_0-rmse:799.86580
[78]	validation_0-rmse:796.04992
[79]	validation_0-rmse:792.25385
[80]	validation_0-rmse:788.47754
[81]	validation_0-rmse:784.72657
[82]	validation_0-rmse:780.98928
[83]	validation_0-rmse:777.27262
[84]	validation_0-rmse:773.57410
[85]	validation_0-rmse:769.89598
[86]	validation_0-rmse:766.23587
[87]	validation_0-rmse:762.59484
[88]	validation_0-rmse:758.97837
[89]	validation_0-rmse:755.37513
[90]	validation_0-rmse:751.79182
[91]	validation_0-rmse:748.22597
[92]	validation_0-rmse:744.67882
[93]	validation_0-rmse:741.15122
[94]	validation_0-rmse:737.64102
[95]	validation_0-rmse:734.15459
[96]	validation_0-rmse:730.68094
[97]	validation_0-rmse:727.22544
[98]	validation_0-rmse:723.78905
[99]	validation_0-rmse:720.36957
[100]	validation_0-rmse:716.97339
[101]	validation_0-rmse:713.58963
[102]	validation_0-rmse:710.22364
[103]	validation_0-rmse:706.87529
[104]	validation_0-rmse:703.54978
[105]	validation_0-rmse:700.23643
[106]	validation_0-rmse:696.94050
[10

...

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

[20:50:36] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1178.95890
[1]	validation_0-rmse:1173.19914
[2]	validation_0-rmse:1167.46878
[3]	validation_0-rmse:1161.76765
[4]	validation_0-rmse:1156.09566
[5]	validation_0-rmse:1150.45262
[6]	validation_0-rmse:1144.83842
[7]	validation_0-rmse:1139.25287
[8]	validation_0-rmse:1133.69583
[9]	validation_0-rmse:1128.16725
[10]	validation_0-rmse:1122.66690
[11]	validation_0-rmse:1117.19465
[12]	validation_0-rmse:1111.75037
[13]	validation_0-rmse:1106.33398
[14]	validation_0-rmse:1100.94525
[15]	validation_0-rmse:1095.58414
[16]	validation_0-rmse:1090.25044
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

[20:50:42] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1179.68362
[1]	validation_0-rmse:1173.91704
[2]	validation_0-rmse:1168.17990
[3]	validation_0-rmse:1162.47206
[4]	validation_0-rmse:1156.79335
[5]	validation_0-rmse:1151.14365
[6]	validation_0-rmse:1145.52278
[7]	validation_0-rmse:1139.93066
[8]	validation_0-rmse:1134.36707
[9]	validation_0-rmse:1128.83196
[10]	validation_0-rmse:1123.32516
[11]	validation_0-rmse:1117.84648
[12]	validation_0-rmse:1112.39585
[13]	validation_0-rmse:1106.97311
[14]	validation_0-rmse:1101.57809
[15]	validation_0-rmse:1096.21069
[16]	validation_0-rmse:1090.87074
[17]	v

C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo_rf.fit(x_treino, y_treino)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f"predicao_xgb {dias_treino} {dias_teste}"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_11856\271300989.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [230]:
metrics = pd.DataFrame(metrics, columns=['Descrição','Métrica', 'XGBoosting','Random Forest','Linear Regression'])
metrics.to_csv('metrica_teste_Intervalos.csv',index=False)
metrics

,Descrição,Métrica,XGBoosting,Random Forest,Linear Regression
0,Teste Intervalos 14 79,RRSE,0.716661,0.775998,1.272785
1,Teste Intervalos 14 79,MAPE,0.105299,0.110862,0.197197
2,Teste Intervalos 14 79,Tempo,0:00:01.731999,0:00:01.306003,0:00:00.002003
3,Teste Intervalos 28 65,RRSE,0.715926,0.715005,1.015333
4,Teste Intervalos 28 65,MAPE,0.104665,0.104939,0.15685
5,Teste Intervalos 28 65,Tempo,0:00:01.895000,0:00:01.687002,0:00:00.001998
6,Teste Intervalos 42 51,RRSE,0.687216,0.72212,0.953827
7,Teste Intervalos 42 51,MAPE,0.107236,0.11155,0.160238
8,Teste Intervalos 42 51,Tempo,0:00:02.304000,0:00:02.112000,0:00:00.003003
9,Teste Intervalos 56 37,RRSE,0.700957,0.714325,0.940524


In [231]:
df_teste.to_csv('previsão_teste_intervalos.csv',index=False)
df_teste.head()

,tempo_viagem,dia_semana,partidaTimeStamp,tipo_dia,turno_dia,hora_dia,data_partida2,predicao_xgb 14 79,predicao_rf 14 79,predicao_lr 14 79,...,predicao_lr 28 65,predicao_xgb 42 51,predicao_rf 42 51,predicao_lr 42 51,predicao_xgb 56 37,predicao_rf 56 37,predicao_lr 56 37,predicao_xgb 70 23,predicao_lr 70 23,predicao_rf 70 23
data_partida,,,,,,,,,,,,,,,,,,,,,
2019-09-10 00:05:40,722,2,1568084740,1,1,0,2019-09-10 00:05:40,861.536194,887.108759,695.343552,...,856.565389,843.985168,869.076898,1004.378035,842.732544,862.750653,981.478063,1069.533813,985.156797,954.402462
2019-09-10 00:16:03,839,2,1568085363,1,1,0,2019-09-10 00:16:03,861.536194,887.108759,695.316902,...,856.555564,843.985168,869.076898,1004.386020,842.732544,862.750653,981.481796,1069.533813,985.160006,954.402462
2019-09-10 00:40:23,746,2,1568086823,1,1,0,2019-09-10 00:40:23,861.536194,887.108759,695.254446,...,856.532541,843.985168,869.076898,1004.404732,842.732544,862.750653,981.490544,1069.533813,985.167526,954.402462
2019-09-10 04:24:52,783,2,1568100292,1,1,4,2019-09-10 04:24:52,810.257690,830.461831,767.163920,...,925.891714,821.052185,863.362345,1075.280632,816.322876,813.946329,1049.680381,1042.127563,1050.572068,913.281469
2019-09-10 05:00:16,780,2,1568102416,1,2,5,2019-09-10 05:00:16,819.322693,859.650436,785.194472,...,943.251112,866.643738,939.007916,1092.983673,866.805298,938.754532,1066.720390,1092.401489,1066.916800,968.265505
